In [6]:
%set_env OP=/cluster/project/infk/hilliges/koenigma/VariTex/outputFolder/
%set_env FP=/cluster/project/infk/hilliges/koenigma/VariTex/
%set_env DP=/cluster/work/hilliges/shared/face_datasets/
%set_env CP=/cluster/project/infk/hilliges/koenigma/VariTex/pretrained/ep44.pkt

env: OP=/cluster/project/infk/hilliges/koenigma/VariTex/outputFolder/
env: FP=/cluster/project/infk/hilliges/koenigma/VariTex/
env: DP=/cluster/work/hilliges/shared/face_datasets/
env: CP=/cluster/project/infk/hilliges/koenigma/VariTex/pretrained/ep44.pkt


In [3]:
import sys
sys.path.append('/cluster/project/infk/hilliges/koenigma/VariTex/')

import json
import os
from tqdm import tqdm

import pytorch_lightning as pl
import torch
import wandb
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
from torch.utils.data import DataLoader

try:
    from varitex.custom_callbacks.callbacks import ImageLogCallback
    from varitex.data.npy_dataset import NPYDataset
    from varitex.modules.pipeline import PipelineModule
    from varitex.options.train_options import TrainOptions
    from mutil.files import copy_src, mkdir
except ModuleNotFoundError:
    print("Have you added VariTex to your pythonpath?")
    print('To fix this error, go to the root path of the repository ".../VariTex/" \n '
          'and run \n'
          "export PYTHONPATH=$PYTHONPATH:$(pwd)")
    exit()


## Workflow

1. Load Data
2. Initialize Latent Space randomly or with PCA using a subset of the training data |Z| = (|data|, |latentCode|)
3. Project to L2 ball
    3. Don't forget to push tensors on Cuda and make z var
4. For all images in training data
    4. Generate Data
    4. Calc loss and backprop
    4. project Z to l2 ball
    

In [5]:
from sklearn.decomposition import PCA

In [6]:

if __name__ == "__main__":    
    """TODO: Some dataset transformations, GLO uses Resize(64),crop(64), toTensor(), normalize((.5,.5,.5),(.5,.5,.5))"""
    if opt.dataset_split == "all":
        # The dataset has no splits or we want to use the full dataset.
        dataset = NPYDataset(opt, split="all", augmentation=True)
        dataloader = DataLoader(dataset, batch_size=opt.batch_size, num_workers=opt.num_workers, shuffle=True)
        do_validation = False
    else:
        # Separate dataloaders for train and validation.
        train_dataset, val_dataset = NPYDataset(opt, split="train", augmentation=True), NPYDataset(opt, split="val",
                                                                                                   augmentation=False)
        train_dataloader = DataLoader(train_dataset, batch_size=opt.batch_size, num_workers=opt.num_workers,
                                      shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=opt.batch_size, num_workers=opt.num_workers, shuffle=False)
        do_validation = True

    ## Define some testing parameters
    """TODO: define other opt file or change it there/make some default params here (plac)"""
    batch_size = 128
    epochs = 25 
    code_dim = 128 #2x128 = 256 (latent code size for varitex)
    lr_z = 10. #Learning rate for the representation space
    loss = 'lap_l1' #Laplacian loss (add combination of l2 loss and lap_l1 here at some point)
    init = 'pca' #or use 'random' to initialize randomly
    n_pca = (64*64*3*2) #Number of samples to be taken for the pca What is the size here? (imgSize^2*|RGB|*2)
        #where does the 2 come from here? 64 can't be from image size
     
        
    if init == 'pca':
        from sklearn.decomposition import PCA
        
        # first, take a subset of train set to fit the PCA
        X_pca = np.vstack([
            X.cpu().numpy().reshape(len(X), -1)
            for i, (X, _, _)
             in zip(tqdm(range(n_pca // train_loader.batch_size), 'collect data for PCA'), 
                    train_loader)
        ])
        print("perform PCA...")
        pca = PCA(n_components=code_dim)
        pca.fit(X_pca)
        # then, initialize latent vectors to the pca projections of the complete dataset
        Z = np.empty((len(train_dataloader.dataset), code_dim))
        for X, _, idx in tqdm(train_loader, 'pca projection'):
            Z[idx] = pca.transform(X.cpu().numpy().reshape(len(X), -1))

    elif init == 'random':
        Z = np.random.randn(len(train_set), code_dim)
        
        
        
    pipeline = PipelineModule(opt)
    gpus = torch.cuda.device_count()
    print("Using {} GPU".format(gpus))
    print("Writing results to {}".format(opt.path_out))
    mkdir(opt.path_out)

    if opt.logger == "wandb":
        wandb.login()
        logger = pl.loggers.WandbLogger(save_dir=opt.path_out, name=opt.experiment_name, project=opt.project)
        logger.log_hyperparams(opt)
        logger.watch(pipeline)
    elif opt.logger == "tensorboard":
        logger = TensorBoardLogger(
            save_dir=opt.path_out, name=opt.experiment_name
        )

    trainer = pl.Trainer(logger, gpus=gpus, max_epochs=opt.max_epochs, default_root_dir=opt.path_out,
                         terminate_on_nan=False,  # Terminate on nan is expensive
                         limit_val_batches=0.25, callbacks=[ImageLogCallback(opt), ModelCheckpoint()],
                         fast_dev_run=opt.debug,
                         resume_from_checkpoint=opt.checkpoint, weights_summary='top')

    if not opt.debug:
        # We keep a copy of the current source code and opt config
        src_path = os.path.dirname(os.path.realpath(__file__))
        copy_src(path_from=src_path,
                 path_to=opt.path_out)
        with open(os.path.join(opt.path_out, "opt.json"), 'w') as f:
            json.dump(opt.__dict__, f)

    if do_validation:
        trainer.fit(pipeline, train_dataloader, val_dataloader)
    else:
        trainer.fit(pipeline, dataloader)


Global seed set to 1234


TypeError: expected str, bytes or os.PathLike object, not NoneType